In [1]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

from torchvision import transforms
from torchvision.models import resnet50

In [29]:
# 커스텀 데이터셋 만들기
class Dataset(Dataset):
    def __init__(self, dir, image_ids, transforms):
        self.dir = dir
        self.transforms = transforms
        
        self.labbels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        image = Image.open(os.path.join(self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)
        if self.transforms is not None:
            image = self.transforms(image)
        return image, target


In [30]:
# 데이터 augumentation
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [32]:
# 사용자 데이터셋 불러오기
trainset = Dataset('data/train', 'data/mnist_train.csv', transforms_train)
testset = Dataset('data/test', 'data/mnist_test.csv', transforms_test)

train_loader(DataLoader(trainset, batch_size=256, num_workers=0))
test_loader(DatLoader(testset, batch_size=32, num_workers=0))

AttributeError: 

In [33]:
# pretrained resnet50 네트워크 불러오기

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = (resnet50(pretrained=True))
        self.classifier = nn.Linear(1000, 26)
    
    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)
        
        return x
    
# GPU 가속화
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = Model().to(device)
print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

# .pth: 저장된 모델의 확장자 (netron 모델의 시각화 앱)

cuda:0
Layer (type:depth-idx)                        Output Shape              Param #
Model                                         --                        --
├─ResNet: 1-1                                 [1, 1000]                 --
│    └─Conv2d: 2-1                            [1, 64, 128, 128]         9,408
│    └─BatchNorm2d: 2-2                       [1, 64, 128, 128]         128
│    └─ReLU: 2-3                              [1, 64, 128, 128]         --
│    └─MaxPool2d: 2-4                         [1, 64, 64, 64]           --
│    └─Sequential: 2-5                        [1, 256, 64, 64]          --
│    │    └─Bottleneck: 3-1                   [1, 256, 64, 64]          75,008
│    │    └─Bottleneck: 3-2                   [1, 256, 64, 64]          70,400
│    │    └─Bottleneck: 3-3                   [1, 256, 64, 64]          70,400
│    └─Sequential: 2-6                        [1, 512, 32, 32]          --
│    │    └─Bottleneck: 3-4                   [1, 512, 32, 32]          

In [34]:
# 학습을 위한 코드
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 100
model.train()

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        
        images = images.to(device)
        targets = targets.to(devie)
        
        outputs = model(images)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f"{epoch}: {loss.item():.5f}, {acc.item():.5f}")

NameError: name 'train_loader' is not defined